In [1]:
import pandas as pd
import numpy as np
from scipy import signal
from tqdm import tqdm
from numpy.fft import fft, fftshift
import random
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import os
from sklearn.preprocessing import StandardScaler
from numpy.random import seed

In [3]:
os.chdir('..')

In [4]:
# data load
# path 변수를 적절히 변경
x_train_path = os.path.join('data/train_features.csv')
y_train_path = os.path.join('data/train_labels.csv')
x_test_path = os.path.join('data/test_features.csv')
sub_path = os.path.join('data/sample_submission.csv')

train = pd.read_csv(x_train_path)
train_label = pd.read_csv(y_train_path)
test = pd.read_csv(x_test_path)
sub = pd.read_csv(sub_path)

In [5]:
train['acc_Energy']=(train['acc_x']**2+train['acc_y']**2+train['acc_z']**2)**(1/3)
test['acc_Energy']=(test['acc_x']**2+test['acc_y']**2+test['acc_z']**2)**(1/3)

train['gy_Energy']=(train['gy_x']**2+train['gy_y']**2+train['gy_z']**2)**(1/3)
test['gy_Energy']=(test['gy_x']**2+test['gy_y']**2+test['gy_z']**2)**(1/3)

train['gy_acc_Energy']=((train['gy_x']-train['acc_x'])**2+(train['gy_y']-train['acc_y'])**2+(train['gy_z']-train['acc_z'])**2)**(1/3)
test['gy_acc_Energy']=((test['gy_x']-test['acc_x'])**2+(test['gy_y']-test['acc_y'])**2+(test['gy_z']-test['acc_z'])**2)**(1/3)

In [6]:
dt=0.02 
def jerk_signal(signal): 
        return np.array([(signal[i+1]-signal[i])/dt for i in range(len(signal)-1)])

In [7]:
train_dt=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    train_dt.append(temp)

100%|██████████████████████████████████████| 3125/3125 [00:27<00:00, 112.52it/s]


In [8]:
test_dt=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for v in train.columns[2:]:
        values=jerk_signal(temp[v].values)
        values=np.insert(values,0,0)
        temp.loc[:,v+'_dt']=values
    test_dt.append(temp)

100%|████████████████████████████████████████| 782/782 [00:06<00:00, 126.14it/s]


In [9]:
from scipy import fftpack
from numpy.fft import *

def fourier_transform_one_signal(t_signal):
    complex_f_signal= fftpack.fft(t_signal)
    amplitude_f_signal=np.abs(complex_f_signal)
    return amplitude_f_signal

In [10]:
train=pd.concat(train_dt)

In [11]:
fft=[]
for i in tqdm(train['id'].unique()):
    temp=train.loc[train['id']==i]
    for i in train.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft.append(temp)
train=pd.concat(fft)

100%|██████████████████████████████████████| 3125/3125 [00:06<00:00, 491.67it/s]


In [12]:
test=pd.concat(test_dt)

In [13]:
fft_t=[]
for i in tqdm(test['id'].unique()):
    temp=test.loc[test['id']==i]
    for i in test.columns[2:8]:
        temp[i]=fourier_transform_one_signal(temp[i].values)
    fft_t.append(temp)
test=pd.concat(fft_t)

100%|████████████████████████████████████████| 782/782 [00:00<00:00, 837.64it/s]


In [14]:
col=train.columns
train_s=train.copy()
test_s=test.copy()

In [15]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

scaler = StandardScaler()

train_s.iloc[:,2:]= scaler.fit_transform(train_s.iloc[:,2:])
train_sc = pd.DataFrame(data = train_s,columns =col)

test_s.iloc[:,2:]= scaler.transform(test_s.iloc[:,2:])
test_sc = pd.DataFrame(data = test_s,columns =col)

In [16]:
# make dataset
# x를 시계열 데이터로 변경 시켜주는 함수
def make_dataset(data):
  ids = data['id'].unique()
  id_data = data.groupby('id')
  series_data = []

  for i in ids:
    df = id_data.get_group(i)
    df = df.drop(['id', 'time'], axis=1)
    series_data.append(df.to_numpy())

  series_data = np.array(series_data)
  return series_data

In [17]:
series_train = make_dataset(train_sc)
series_test = make_dataset(test_sc)

In [18]:
series_train.shape, series_test.shape

((3125, 600, 18), (782, 600, 18))

In [19]:
# label one-hot 인코딩
labels = train_label['label'].to_numpy()

In [20]:
import tensorflow as tf

2021-09-30 09:05:04.933536: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [21]:
# dataset과 validation set을 만들어 주는 함수
# validation set은 shuffle 적용 x
def make_train(series_data, labels):
  cat_y = tf.keras.utils.to_categorical(labels)

  BATCH_SIZE = 64
  train_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
  train_dataset = train_dataset.batch(BATCH_SIZE)
  train_dataset = train_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_dataset

def make_val(series_data, labels):
  cat_y = tf.keras.utils.to_categorical(labels)

  BATCH_SIZE = 64
  val_dataset = tf.data.Dataset.from_tensor_slices((series_data, cat_y))
  val_dataset = val_dataset.batch(BATCH_SIZE)
  val_dataset = val_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

  return train_dataset

In [22]:
from tensorflow import keras
from tensorflow.keras import backend as K

In [23]:
# 가중치 적용 loss
# https://openaccess.thecvf.com/content_CVPR_2019/papers/Cui_Class-Balanced_Loss_Based_on_Effective_Number_of_Samples_CVPR_2019_paper.pdf
# 위 논문 참고
def weighted_loss(labels, y_pred, beta=0.9):
        no_samples = [12, 21, 20, 23, 35, 25, 24, 26, 97, 37, 20, 23, 12,
                     12, 25, 25, 22, 27, 47, 20, 26, 27, 19, 20, 35, 24,
                     1518, 34, 55, 20, 35, 20, 18, 20, 22, 30, 28, 35, 20,
                     20, 34, 20, 20, 35, 21, 22, 20, 26, 25, 30, 37, 24,
                     12, 13, 23, 37, 36, 20, 20, 23, 48]

        weight = tf.divide(1-beta, 1-tf.pow(beta, no_samples))
        #y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        loss = labels * K.log(y_pred) * weight
        loss = -K.sum(loss, -1)
        return loss

# 모델을 만들어 주는 함수
def base():
    seed(2021)
    tf.random.set_seed(2021)
    model = keras.models.Sequential([
            keras.layers.Conv1D(filters=128, kernel_size=9, padding='same', input_shape=[600, 18]),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.3),
            keras.layers.Conv1D(filters=256, kernel_size=6, padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.4),
            keras.layers.Conv1D(filters=128, kernel_size=3,padding='same'),
            keras.layers.BatchNormalization(),
            keras.layers.Activation('relu'),
            keras.layers.Dropout(0.5),
            keras.layers.GlobalAveragePooling1D(),
            keras.layers.Dense(61, activation='softmax')
    ])
    model.compile(optimizer='adam',
                loss='categorical_crossentropy', 
                metrics=['accuracy'])
    return model

In [24]:
def make_callbacks(i):
    # checkpoint path
    # 중간중간 모델의 weight를 저장할 경로 설정
    ckpt_name = str(i) + '_fold_cnn_weighted_ckpt.hdf5'
    checkpoint_dir_path = os.path.join('checkpoint/best/k_fold_cnn')
    checkpoint_path = os.path.join(checkpoint_dir_path, ckpt_name)
    
    # callback 함수 목록
    callbacks_list = [
        # 매 epoch 마다 val_loss를 체크하여 가장 낮은 상태의 weight를 저장
        tf.keras.callbacks.ModelCheckpoint(
            filepath = checkpoint_path,
            monitor='val_loss',
            mode='min',
            save_weights_only=True,
            save_best_only=True
        ),
        # 8번 동안 val_loss의 향상이 없으면 훈련 종료
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            mode='min',
            verbose=1, 
            patience=8
        ),
        tf.keras.callbacks.ReduceLROnPlateau(patience = 4,verbose = 1,factor = 0.5) 
    ]
    
    return callbacks_list, checkpoint_path

In [25]:
from sklearn.model_selection import StratifiedKFold

In [26]:
# k-fold 적용
# 데이터 수가 적어 10개의 fold 사용
k = 10 
split = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
results = []    # 매 fold 훈련 후 loss 저장
models = []     # 매 fold 훈련 후 해당 모델 저장
accs = []
for i, (train, val) in enumerate(split.split(series_train, labels)):
    callbacks_list, checkpoint_path = make_callbacks(i)
    print('Fold', i)
    print('#'*20)

    train_dataset = make_train(series_train[train], labels[train])
    val_dataset = make_val(series_train[val], labels[val])

    model = base()

    model.fit(train_dataset, validation_data=val_dataset, callbacks=callbacks_list, epochs=100)

    # 모델 복원
    model.load_weights(checkpoint_path)
    result = model.evaluate(val_dataset)[0]
    acc = model.evaluate(val_dataset)[1]
    
    results.append(result)
    accs.append(acc)
    models.append(model)

    del model, train_dataset, val_dataset

Fold 0
####################


2021-09-30 09:05:05.556672: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-09-30 09:05:05.586903: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-09-30 09:05:05.587585: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: TITAN RTX computeCapability: 7.5
coreClock: 1.77GHz coreCount: 72 deviceMemorySize: 23.65GiB deviceMemoryBandwidth: 625.94GiB/s
2021-09-30 09:05:05.587608: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2021-09-30 09:05:05.588678: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-09-30 09:05:05.589518: I tensorflow/stream_executor/platform/default/

Epoch 1/100


2021-09-30 09:05:07.039215: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2021-09-30 09:05:07.172616: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7


44/44 [==============================] - 1s 34ms/step - loss: 2.8274 - accuracy: 0.4584 - val_loss: 3.3930 - val_accuracy: 0.2717
Epoch 2/100
44/44 [==============================] - 1s 29ms/step - loss: 1.9542 - accuracy: 0.5452 - val_loss: 1.8818 - val_accuracy: 0.5405
Epoch 3/100
44/44 [==============================] - 1s 27ms/step - loss: 1.7250 - accuracy: 0.5786 - val_loss: 1.9384 - val_accuracy: 0.5505
Epoch 4/100
44/44 [==============================] - 1s 28ms/step - loss: 1.5330 - accuracy: 0.6177 - val_loss: 1.7723 - val_accuracy: 0.5644
Epoch 5/100
44/44 [==============================] - 1s 28ms/step - loss: 1.3786 - accuracy: 0.6422 - val_loss: 1.5702 - val_accuracy: 0.5982
Epoch 6/100
44/44 [==============================] - 1s 29ms/step - loss: 1.2451 - accuracy: 0.6728 - val_loss: 1.3723 - val_accuracy: 0.6309
Epoch 7/100
44/44 [==============================] - 1s 29ms/step - loss: 1.1253 - accuracy: 0.7020 - val_loss: 1.2451 - val_accuracy: 0.6597
Epoch 8/100
44/44 

...

44/44 [==============================] - 1s 29ms/step - loss: 1.0337 - accuracy: 0.7256 - val_loss: 1.0773 - val_accuracy: 0.7017
Epoch 9/100
44/44 [==============================] - 1s 28ms/step - loss: 0.9497 - accuracy: 0.7487 - val_loss: 0.9943 - val_accuracy: 0.7316
Epoch 10/100
44/44 [==============================] - 1s 28ms/step - loss: 0.8723 - accuracy: 0.7640 - val_loss: 0.9228 - val_accuracy: 0.7522
Epoch 11/100
44/44 [==============================] - 1s 27ms/step - loss: 0.8087 - accuracy: 0.7792 - val_loss: 0.8587 - val_accuracy: 0.7736
Epoch 12/100
44/44 [==============================] - 1s 28ms/step - loss: 0.7531 - accuracy: 0.7942 - val_loss: 0.7866 - val_accuracy: 0.7927
Epoch 13/100
44/44 [==============================] - 1s 28ms/step - loss: 0.7019 - accuracy: 0.8077 - val_loss: 0.7426 - val_accuracy: 0.7999
Epoch 14/100
44/44 [==============================] - 1s 29ms/step - loss: 0.6578 - accuracy: 0.8180 - val_loss: 0.6980 - val_accuracy: 0.8006
Epoch 15/100


Epoch 63/100
44/44 [==============================] - 1s 32ms/step - loss: 0.0799 - accuracy: 0.9908 - val_loss: 0.1517 - val_accuracy: 0.9577
Epoch 64/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0772 - accuracy: 0.9900 - val_loss: 0.1406 - val_accuracy: 0.9641
Epoch 65/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0766 - accuracy: 0.9915 - val_loss: 0.1412 - val_accuracy: 0.9623
Epoch 66/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0757 - accuracy: 0.9922 - val_loss: 0.1387 - val_accuracy: 0.9634
Epoch 67/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0740 - accuracy: 0.9922 - val_loss: 0.1397 - val_accuracy: 0.9645
Epoch 68/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0723 - accuracy: 0.9929 - val_loss: 0.1387 - val_accuracy: 0.9627
Epoch 69/100
44/44 [==============================] - 1s 29ms/step - loss: 0.0712 - accuracy: 0.9922 - val_loss: 0.1337 - val_accuracy: 0.9652

Epoch 70/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0712 - accuracy: 0.9947 - val_loss: 0.1444 - val_accuracy: 0.9620
Epoch 71/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0708 - accuracy: 0.9936 - val_loss: 0.1390 - val_accuracy: 0.9637
Epoch 72/100
44/44 [==============================] - 1s 29ms/step - loss: 0.0688 - accuracy: 0.9950 - val_loss: 0.1374 - val_accuracy: 0.9645
Epoch 73/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0690 - accuracy: 0.9936 - val_loss: 0.1417 - val_accuracy: 0.9623
Epoch 74/100
44/44 [==============================] - 1s 29ms/step - loss: 0.0667 - accuracy: 0.9950 - val_loss: 0.1357 - val_accuracy: 0.9637
Epoch 75/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0676 - accuracy: 0.9950 - val_loss: 0.1366 - val_accuracy: 0.9652
Epoch 76/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0660 - accuracy: 0.9954 - val_loss: 0.1383 - val_accuracy: 0.9630

44/44 [==============================] - 1s 28ms/step - loss: 0.0308 - accuracy: 0.9982 - val_loss: 0.1186 - val_accuracy: 0.9726
Epoch 80/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0281 - accuracy: 0.9986 - val_loss: 0.1079 - val_accuracy: 0.9744
Epoch 81/100
43/44 [============================>.] - ETA: 0s - loss: 0.0281 - accuracy: 0.9982
Epoch 00081: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
44/44 [==============================] - 1s 27ms/step - loss: 0.0281 - accuracy: 0.9982 - val_loss: 0.1040 - val_accuracy: 0.9769
Epoch 82/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0285 - accuracy: 0.9979 - val_loss: 0.1143 - val_accuracy: 0.9705
Epoch 83/100
44/44 [==============================] - 1s 27ms/step - loss: 0.0275 - accuracy: 0.9979 - val_loss: 0.1123 - val_accuracy: 0.9712
Epoch 84/100
44/44 [==============================] - 1s 28ms/step - loss: 0.0274 - accuracy: 0.9979 - val_loss: 0.1154 - val_accuracy:

44/44 [==============================] - 1s 27ms/step - loss: 1.2444 - accuracy: 0.6637 - val_loss: 1.3848 - val_accuracy: 0.6282
Epoch 7/100
44/44 [==============================] - 1s 29ms/step - loss: 1.1290 - accuracy: 0.6918 - val_loss: 1.2286 - val_accuracy: 0.6612
Epoch 8/100
44/44 [==============================] - 1s 28ms/step - loss: 1.0337 - accuracy: 0.7220 - val_loss: 1.0864 - val_accuracy: 0.7007
Epoch 9/100
44/44 [==============================] - 1s 28ms/step - loss: 0.9489 - accuracy: 0.7430 - val_loss: 0.9869 - val_accuracy: 0.7273
Epoch 10/100
44/44 [==============================] - 1s 29ms/step - loss: 0.8766 - accuracy: 0.7590 - val_loss: 0.9229 - val_accuracy: 0.7455
Epoch 11/100
44/44 [==============================] - 1s 30ms/step - loss: 0.8100 - accuracy: 0.7803 - val_loss: 0.8465 - val_accuracy: 0.7632
Epoch 12/100
44/44 [==============================] - 1s 29ms/step - loss: 0.7546 - accuracy: 0.7863 - val_loss: 0.7918 - val_accuracy: 0.7800
Epoch 13/100
44

In [27]:
results

[0.1491401046514511,
 0.10602536797523499,
 0.103817418217659,
 0.09942291676998138,
 0.12268910557031631,
 0.1125587746500969,
 0.11477942019701004,
 0.1015380248427391,
 0.12599708139896393,
 0.12447932362556458]

In [31]:
accs

[0.9601706862449646,
 0.9747510552406311,
 0.9690611958503723,
 0.9743954539299011,
 0.9679943323135376,
 0.9736935496330261,
 0.9726271033287048,
 0.9779594540596008,
 0.970138669013977,
 0.9697831273078918]

In [28]:
# 결과 생성
pred_list = []    # 예측 결과를 담을 리스트
for model in models:
  pred = model.predict(series_test)
  pred_list.append(pred)

pred = np.mean(pred_list, axis=0)

In [29]:
# 제출물 생성
sub.iloc[:, 1:] = pred

save_path = os.path.join('sub/test.csv')     # 저장 경로
sub.to_csv(save_path, index=False)